In [1]:
import numpy as np
import cv2

## Zadanie 1 (popraw parametry)

In [25]:
def median(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    shape = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start)).shape
    BUF = np.zeros((shape[0],shape[1],60),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.median(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Median")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

def average(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    shape = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start)).shape
    BUF = np.zeros((shape[0],shape[1],60),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.average(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Average:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [ ]:
median("highway", 1, 36, 3, 5, 11)
average("highway", 1, 36, 5, 3, 11)

Median
P =  0.5895435339622598
R =  0.4632446879873348
F1 =  0.5188183240491713
Average:
P =  0.40449940769410647
R =  0.3857284779819408
F1 =  0.39489100220998086


In [31]:
median("office", 1, 6, 3, 4, 9)
average("office", 1, 6, 3, 4, 9)

Median
P =  0.3987271033139201
R =  0.5311097055875579
F1 =  0.45549462534403495
Average:
P =  0.340482856477874
R =  0.6033027412106371
F1 =  0.4352985278678191


In [26]:
median("pedestrian", 5, 16, 3, 4, 11)
average("pedestrian", 5, 16, 3, 4, 11)

Median
P =  0.04917520171797554
R =  0.9957116235209592
F1 =  0.09372176728973379
Average:
P =  0.02784166593640009
R =  0.9962878974980729
F1 =  0.054169542231742335


## Zadanie 2

In [35]:
def median_nonbuf(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    I = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    I_2 = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start+10))
    I_2 = cv2.cvtColor(I_2, cv2.COLOR_BGR2GRAY)

    I_diff = cv2.absdiff(I,I_2)
    B = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
    B = B[1]
    mask = B[1]

    for i in range(roi_start+2, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        #mask = np.where((mask == I), mask, np.where((mask < I), mask + 1, np.where(mask > I, mask-1, mask))) #Liberalne
        mask = np.where(B == 255, np.where((mask == I), mask,
                   np.where((mask < I), mask + 1, mask - 1)), mask)
        I_diff = cv2.absdiff(I, mask)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        B = thresh
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Non-buffered median:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

def average_nonbuf(name, speed, threshold_value, kernel_1, kernel_2, blur, alpha):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    mask = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.float64)
    f.close()

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        mask = (alpha*I + (1-alpha)*mask).astype(np.float64)
        I_diff = cv2.absdiff(I, mask.astype(np.uint8))

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Non-buffered Average:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [37]:
median_nonbuf("highway", 1, 36, 3, 5, 9)

Non-buffered median:
P =  0.3631974708968293
R =  0.8395807503434555
F1 =  0.5070487638593304


In [12]:
average_nonbuf("highway", 5, 36, 5, 5, 9, 0.05)

Non-buffered Average:
P =  0.932500505161182
R =  0.602385970976018
F1 =  0.7319436726692932


In [15]:
median_nonbuf("office", 5, 6, 3, 4, 9)
average_nonbuf("office", 5, 6, 3, 4, 9, 0.05)

Non-buffered median:
P =  0.11668473870376297
R =  0.945720153868323
F1 =  0.20773832992023855
Non-buffered Average:
P =  0.3487055514795794
R =  0.8251625750663639
F1 =  0.4902403673664525


In [17]:
median_nonbuf("pedestrian", 5, 16, 3, 5, 15)
average_nonbuf("pedestrian", 5, 16, 3, 4, 15, 0.05)

Non-buffered median:
P =  0.045113128240000105
R =  0.9969988848724339
F1 =  0.08632035324888736
Non-buffered Average:
P =  0.8001244064518911
R =  0.9723089127880434
F1 =  0.8778531562090356


## Zadanie 4

In [45]:
def mog(name, speed, threshold_value, kernel_1, kernel_2, blur):
    mog = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=36, detectShadows=True)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    mask = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.float64)
    f.close()

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        mask = mog.apply(I)

        (T,thresh) = cv2.threshold(mask,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Non-buffered Average:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [46]:
mog("highway", 5, 36, 3, 4, 11)

Non-buffered Average:
P =  0.9164095840272961
R =  0.8320232677212763
F1 =  0.8721800164197916


In [47]:
mog("office", 5, 6, 3, 4, 9)

Non-buffered Average:
P =  0.5911209256162017
R =  0.0996093105729205
F1 =  0.17048956243964045


In [49]:
mog("pedestrian", 5, 16, 3, 4, 11)

Non-buffered Average:
P =  0.5514579795996613
R =  0.9795759521617758
F1 =  0.7056603570138503


## Zadanie 5

In [5]:
def knn(name, speed, threshold_value, kernel_1, kernel_2, blur):
    knn = cv2.createBackgroundSubtractorKNN(history=100, detectShadows=True)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    mask = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.float64)
    f.close()

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        mask = knn.apply(I)

        (T,thresh) = cv2.threshold(mask,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Non-buffered Average:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [6]:
knn("highway", 5, 36, 3, 4, 11)

Non-buffered Average:
P =  0.7671706318350323
R =  0.9441251118021728
F1 =  0.8464989891380763


In [7]:
knn("office", 5, 6, 3, 4, 9)

Non-buffered Average:
P =  0.525060380662792
R =  0.25262634615974455
F1 =  0.3411247251757566


In [8]:
knn("pedestrian", 5, 16, 3, 4, 11)

Non-buffered Average:
P =  0.26906201850585415
R =  0.9775777034359401
F1 =  0.4219808265423973
